In [1]:
import pandas as pd
from scipy.sparse import hstack                               #Для объединения матриц
from sklearn.feature_extraction.text import TfidfVectorizer  #TF-IDF
from sklearn.linear_model import Ridge                      # гребневая регрессия
from sklearn.feature_extraction import DictVectorizer      #LocationNormalized и ContractTime являются строковыми, 
                                                            # и поэтому с ними нельзя работать напрямую. 
                                                              # Разбиваем категориальные на m бинарных
data = pd.read_csv('salary-train.csv')
test = pd.read_csv('salary-test-mini.csv')
df_train = pd.DataFrame(data, columns  = ['FullDescription', 'LocationNormalized', 'ContractTime',  'SalaryNormalized' ])
df_test  = pd.DataFrame(test, columns  = ['FullDescription', 'LocationNormalized', 'ContractTime',  'SalaryNormalized' ])
df_train['FullDescription'] = df_train['FullDescription'].str.lower()                        #Приведение к нижнему регистру
df_test['FullDescription'] = df_test['FullDescription'].str.lower()                        
df_train['FullDescription'] = df_train['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True) # Замена всего, кроме букв и цифр на пробелы
df_test['FullDescription'] = df_test['FullDescription'].replace('[^a-zA-Z0-9]', ' ', regex = True) #

In [2]:
vectorizer = TfidfVectorizer(min_df=5)  # оставляем те слова, которые встречаются хоты бы на 5 объектах

X_train_text = vectorizer.fit_transform(df_train['FullDescription']) #преобразование текстов в векторы признаков
X_test_text =  vectorizer.transform(df_test['FullDescription'])       # на выходе получаем разреженные матрицы

df_train['LocationNormalized'].fillna('nan', inplace=True)         #замена пропущенных значений 
df_train['ContractTime'].fillna('nan', inplace=True)               # на специальные строковые величины
df_test['LocationNormalized'].fillna('nan', inplace=True)    
df_test['ContractTime'].fillna('nan', inplace=True)

In [5]:
enc = DictVectorizer()

X_train_categ = enc.fit_transform(df_train[['LocationNormalized', 'ContractTime']].to_dict('records'))  # обучаем и получаем разреженную матрицу
X_test_categ = enc.transform(df_test[['LocationNormalized', 'ContractTime']].to_dict('records'))  # аппроксимируем по обученной

train_data = hstack([X_train_text,X_train_categ])   #Ибо не нужно пытаться обсчитать полноценный массив, когда есть разреженная матрица
test_data =  hstack([X_test_text,X_test_categ])
train_target = df_train['SalaryNormalized']
test_target = df_test['SalaryNormalized']

In [6]:
predict = Ridge (alpha = 1.0, random_state = 241)
predict.fit(train_data,train_target)                 #обучаем модель

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=241, solver='auto', tol=0.001)

In [10]:
print(predict.predict(test_data))

[56555.61500155 37188.32442618]
